In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

In [3]:


class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            super().__init__()
            self.K = param_K  # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*10
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        if session.short_order_size or session.long_position_size: #全開の注文処理中の場合はなにもしない       
            return 
        
        ohlcv_df = session.ohlcv(60*60*2, 6)       # 2時間足(60*60*2sec)を６本取得。 最新は６番目。ただし未確定足
        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean() * self.K     #　過去５本足のレンジの平均値 * K

        # Long/Short判定
        diff_low   =   (ohlcv_latest["open"][0] - ohlcv_latest["low"][0])
        detect_short  = range_width < diff_low

        diff_high  = - (ohlcv_latest["open"][0] - ohlcv_latest["high"][0])  
        detect_long = range_width  < diff_high
        
        self.log_indicator('diff_low', time_us, diff_low)
        self.log_indicator('diff_high', time_us, diff_high)
        self.log_indicator('range', time_us, range_width)

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_order_size:
            #    print("makeorder")                                    
                if session.short_position_size:
                    session.place_order("Buy", session.best_buy_price, 0.02, 60*30, "doten Long")                        
                else:
                    session.place_order("Buy", session.best_buy_price, 0.01, 60*30, "Open Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.long_order_size:
                #print("makeorder")                                                      
                if session.long_position_size:
                    session.place_order("Sell", session.best_sell_price, 0.02, 60*30, "Doten Short")                     
                else:
                    session.place_order("Sell", session.best_sell_price, 0.01, 60*30, "Open Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [4]:
back_tester = BackTester("BN", "BTCBUSD", False)
back_tester.maker_fee_rate = 0.1 * 0.01  # 0.1%
back_tester.maker_fee_rate

0.001

In [5]:
Market.download(10)

agent = Agent()

r = back_tester.run(agent, rbot.DAYS_BEFORE(30), 0)

r



,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-11-13 07:10:00+00:00,2022-11-13 07:10:00.910000+00:00,0000-0001,0,Sell,True,Open,16626.53,0.01,166.2653,0.00,...,16626.53,0.01,166.2653,0.0000,0.166265,-0.166265,-0.01,Open Short,-0.166265,-0.01
2022-11-14 06:50:00+00:00,2022-11-14 07:20:00.036000+00:00,0000-0002,0,Buy,True,Expire,0.00,0.00,0.0000,0.00,...,15965.53,0.02,319.3106,0.0000,0.000000,0.000000,0.00,doten Long,-0.166265,-0.01
2022-11-14 07:40:00+00:00,2022-11-14 07:39:59.924000+00:00,0000-0003,0,Buy,True,Close,16626.53,0.01,166.2653,16746.46,...,16746.46,0.01,167.4646,-1.1993,0.167465,-1.366765,0.01,doten Long,-1.533030,0.00
2022-11-14 07:40:00+00:00,2022-11-14 07:39:59.924000+00:00,0000-0003,1,Buy,True,Open,16746.46,0.01,167.4646,0.00,...,16746.46,0.01,167.4646,0.0000,0.167465,-0.167465,0.01,doten Long,-1.700494,0.01
2022-11-15 19:30:00+00:00,2022-11-15 19:30:00.305000+00:00,0000-0004,0,Sell,True,Close,16746.46,0.01,167.4646,16765.00,...,16765.00,0.01,167.6500,0.1854,0.167650,0.017750,-0.01,Doten Short,-1.682744,0.00
2022-11-15 19:30:00+00:00,2022-11-15 19:30:00.305000+00:00,0000-0004,1,Sell,True,Open,16765.00,0.01,167.6500,0.00,...,16765.00,0.01,167.6500,0.0000,0.167650,-0.167650,-0.01,Doten Short,-1.850394,-0.01
2022-11-17 15:50:00+00:00,2022-11-17 15:49:59.996000+00:00,0000-0005,0,Buy,True,Close,16765.00,0.01,167.6500,16637.36,...,16637.36,0.01,166.3736,1.2764,0.166374,1.110026,0.01,doten Long,-0.740368,0.00
2022-11-17 15:50:00+00:00,2022-11-17 15:49:59.996000+00:00,0000-0005,1,Buy,True,Open,16637.36,0.01,166.3736,0.00,...,16637.36,0.01,166.3736,0.0000,0.166374,-0.166374,0.01,doten Long,-0.906742,0.01
2022-11-18 15:50:00+00:00,2022-11-18 15:50:04.601000+00:00,0000-0006,0,Sell,True,Close,16637.36,0.01,166.3736,16679.34,...,16679.34,0.01,166.7934,0.4198,0.166793,0.253007,-0.01,Doten Short,-0.653735,0.00


In [6]:
market = Market.open("BN", "BTCBUSD")
ohlcv = market.ohlcv(rbot.NOW()-rbot.DAYS(30), 0, 60*60)


In [7]:
back_tester

Exchange name,BN
Market name,BTCBUSD
Size in price currency,False
maker fee rate,0.1 [%]
Agent class name,Agent
enable: on_tick,False
enable: on_clock,True
enable: on_update,True
clock interval:,600 [sec]
start,"2022-11-13T07:04:14.416000 (1,668,323,054,416,000)"
end,"2022-12-10T23:59:59.943000 (1,670,716,799,943,000)"


In [8]:
from collections import OrderedDict
import numpy as np


from bokeh.layouts import column 
from bokeh.models import ColumnDataSource, RangeTool, HoverTool
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
import datetime




In [10]:
df[(df['order_side'] == "Buy") & (df['status'] == 'Expire') & (df['sub_id'] == 0)]


NameError: name 'df' is not defined

In [ ]:
df[(df['order_side'] == "Sell") & (df['status'] == 'Expire') & (df['sub_id'] == 0)]

In [ ]:
df[(df['order_side'] == "Buy") & (df['status'] != 'Expire') & (df['sub_id'] == 0)]

In [ ]:
df[(df['order_side'] == "Sell") & (df['status'] != 'Expire') & (df['sub_id']== 0)]


In [9]:
def draw_order_maker(p, df, marker, fill_color, line_color, legend_name):
    df = ColumnDataSource(df)
    scatter = p.scatter(x='update_time', y='order_price', source=df, marker=marker, size=12, fill_color=fill_color, line_width=0, legend_label=legend_name)

    hover = HoverTool(
        renderers=[scatter],
        tooltips = [
            (legend_name,""),
            ("timestamp", "@update_time{%F %R.%S}"),
            ("Price", "@order_price{0.0}")
        ],
        formatters={
            "@update_time": "datetime"
        }
    )
    p.add_tools(hover_buy)    

In [11]:
df = r

chart = Chart(900, 400, ohlcv)

p = chart.new_figure('profit', 150, 'profit')
chart.step(p, df, 'sum_profit', legend_label='profit', color="#ff8080")

p = chart.get_figure('price')
#buy_df = ColumnDataSource(df[(df['order_side'] == "Buy") & (df['status'] != 'Expire')])
#scatter = p.scatter(x=buy_df['update_time'], y=buy_df['order_price'], marker='triangle', size=10, fill_color="#0000ff60", line_width=0, legend_label="Buy")





buy_df = ColumnDataSource(df[(df['order_side'] == "Buy") & (df['status'] != 'Expire') & (df['sub_id'] == 0)])
buy_scatter = p.scatter(x='update_time', y='order_price', source=buy_df, marker='triangle', size=12, fill_color="#0000ff", line_width=0, legend_label="Buy")

hover_buy = HoverTool(
    renderers=[buy_scatter],
    tooltips = [
        ("Buy Order",""),
        ("timestamp", "@update_time{%F %R.%S}"),
        ("Price", "@order_price{0.0}")
    ],
    formatters={
        "@update_time": "datetime"
    }
)
p.add_tools(hover_buy)


sell_df = ColumnDataSource(df[(df['order_side'] == "Sell") & (df['status'] != 'Expire') & (df['sub_id']== 0)])
sell_scatter = p.scatter(x='update_time', y='order_price', source=sell_df, marker='inverted_triangle', size=12, fill_color="#ff000060", line_width=0, legend_label='Sell')

hover_sell = HoverTool(
    renderers=[sell_scatter],
    tooltips = [
        ("Sell Order",""),
        ("timestamp", "@update_time{%F %R.%S}"),
        ("Price", "@order_price{0.0}")
    ],
    formatters={
        "@update_time": "datetime"
    }
)
p.add_tools(hover_sell)


buy_df_e = ColumnDataSource(df[(df['order_side'] == "Buy") & (df['status'] == 'Expire') & (df['sub_id'] == 0)])
p.scatter(x='index', y='order_price', marker='triangle', source=buy_df_e, size=32, fill_color="#00000000", line_width=2, line_color="#0000ff", legend_label='Buy Expire')

sell_df_e = ColumnDataSource(df[(df['order_side'] == "Sell") & (df['status'] == 'Expire') & (df['sub_id'] == 0)])
p.scatter(x='index', y='order_price', source=sell_df_e, marker='inverted_triangle', size=32, fill_color="#00000000", line_width=2, line_color="#ff0000", legend_label='Sell Expire')



chart.new_figure('fig_position', 100, 'position')
chart.step('fig_position', df, 'position', legend_label='position', color="#80ff80")

chart.new_figure('fig_indicator', 200, 'indicator')
indicator = agent.get_indicator('diff_low')
chart.line('fig_indicator', indicator, 'value', legend_label='diff_low', color='#ff8080')

indicator = agent.get_indicator('diff_high')
chart.line('fig_indicator', indicator, 'value', legend_label='diff_high', color='#80ff80')

indicator = agent.get_indicator('range')
chart.line('fig_indicator', indicator, 'value', legend_label='range', color='#8080ff')

chart.show()

Loading BokehJS ...

price
volume
profit
fig_position
fig_indicator


In [73]:
df[(df['order_side'] == "Sell") & (df['status'] != 'Expire') & (df['sub_id'] == 0)]
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-11-13 06:50:00+00:00,2022-11-13 07:20:00.010000+00:00,0000-0001,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,16785.95,0.01,167.8595,0.0000,0.000000,0.000000,0.00,Open Short,0.000000,0.00
2022-11-13 07:40:00+00:00,2022-11-13 07:40:00.086000+00:00,0000-0002,0,Sell,True,Open,16666.34,0.01,166.6634,0.00,...,16666.34,0.01,166.6634,0.0000,0.166663,-0.166663,-0.01,Open Short,-0.166663,-0.01
2022-11-14 06:50:00+00:00,2022-11-14 07:20:00.036000+00:00,0000-0003,0,Buy,True,Expire,0.00,0.00,0.0000,0.00,...,15965.53,0.02,319.3106,0.0000,0.000000,0.000000,0.00,doten Long,-0.166663,-0.01
2022-11-14 07:40:00+00:00,2022-11-14 07:39:59.924000+00:00,0000-0004,0,Buy,True,Close,16666.34,0.01,166.6634,16746.46,...,16746.46,0.01,167.4646,-0.8012,0.167465,-0.968665,0.01,doten Long,-1.135328,0.00
2022-11-14 07:40:00+00:00,2022-11-14 07:39:59.924000+00:00,0000-0004,1,Buy,True,Open,16746.46,0.01,167.4646,0.00,...,16746.46,0.01,167.4646,0.0000,0.167465,-0.167465,0.01,doten Long,-1.302793,0.01
2022-11-15 19:30:00+00:00,2022-11-15 19:30:00.305000+00:00,0000-0005,0,Sell,True,Close,16746.46,0.01,167.4646,16765.00,...,16765.00,0.01,167.6500,0.1854,0.167650,0.017750,-0.01,Doten Short,-1.285043,0.00
2022-11-15 19:30:00+00:00,2022-11-15 19:30:00.305000+00:00,0000-0005,1,Sell,True,Open,16765.00,0.01,167.6500,0.00,...,16765.00,0.01,167.6500,0.0000,0.167650,-0.167650,-0.01,Doten Short,-1.452693,-0.01
2022-11-17 15:30:00+00:00,2022-11-17 15:48:07.242000+00:00,0000-0006,0,Buy,True,Close,16765.00,0.01,167.6500,16595.58,...,16595.58,0.01,165.9558,1.6942,0.165956,1.528244,0.01,doten Long,0.075552,0.00
2022-11-17 15:30:00+00:00,2022-11-17 15:48:07.242000+00:00,0000-0006,1,Buy,True,Open,16595.58,0.01,165.9558,0.00,...,16595.58,0.01,165.9558,0.0000,0.165956,-0.165956,0.01,doten Long,-0.090404,0.01


In [15]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-02 13:40:00+00:00,2022-12-02 14:10:00.053000+00:00,0000-0001,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,17026.48,0.01,170.2648,0.0000,0.000000,0.000000,0.00,Open Short,0.000000,0.00
2022-12-03 21:50:00+00:00,2022-12-03 21:50:00.015000+00:00,0000-0002,0,Sell,True,Open,16924.43,0.01,169.2443,0.00,...,16924.43,0.01,169.2443,0.0000,0.169244,-0.169244,-0.01,Open Short,-0.169244,-0.01
2022-12-04 01:30:00+00:00,2022-12-04 01:52:42.255000+00:00,0000-0003,0,Buy,True,Close,16924.43,0.01,169.2443,16962.71,...,16962.71,0.01,169.6271,-0.3828,0.169627,-0.552427,0.01,doten Long,-0.721671,0.00
2022-12-04 01:30:00+00:00,2022-12-04 01:52:42.255000+00:00,0000-0003,1,Buy,True,Open,16962.71,0.01,169.6271,0.00,...,16962.71,0.01,169.6271,0.0000,0.169627,-0.169627,0.01,doten Long,-0.891298,0.01
2022-12-05 15:10:00+00:00,2022-12-05 15:40:00.146000+00:00,0000-0004,0,Sell,True,Expire,0.00,0.00,0.0000,0.00,...,17207.99,0.02,344.1598,0.0000,0.000000,0.000000,0.00,Doten Short,-0.891298,0.01
2022-12-05 16:00:00+00:00,2022-12-05 16:08:35.636000+00:00,0000-0005,0,Sell,True,Close,16962.71,0.01,169.6271,17088.30,...,17088.30,0.01,170.8830,1.2559,0.170883,1.085017,-0.01,Doten Short,0.193719,0.00
2022-12-05 16:00:00+00:00,2022-12-05 16:08:35.636000+00:00,0000-0005,1,Sell,True,Open,17088.30,0.01,170.8830,0.00,...,17088.30,0.01,170.8830,0.0000,0.170883,-0.170883,-0.01,Doten Short,0.022836,-0.01
2022-12-06 23:40:00+00:00,2022-12-06 23:42:12.620000+00:00,0000-0006,0,Buy,True,Close,17088.30,0.01,170.8830,17067.54,...,17067.54,0.01,170.6754,0.2076,0.170675,0.036925,0.01,doten Long,0.059760,0.00
2022-12-06 23:40:00+00:00,2022-12-06 23:42:12.620000+00:00,0000-0006,1,Buy,True,Open,17067.54,0.01,170.6754,0.00,...,17067.54,0.01,170.6754,0.0000,0.170675,-0.170675,0.01,doten Long,-0.110915,0.01


In [ ]:
back_tester.last_run_record

17882387

In [ ]:
back_tester.last_run_start

1669956189170000

In [ ]:
len(back_tester.result)

9

In [10]:
chart.figure

NameError: name 'chart' is not defined

In [14]:
from bokeh.models.glyphs import Step
from bokeh.models import ColumnDataSource

chart2 = Chart(800, 400, ohlcv)
p = chart2.new_figure('profit', 100, 'profit')
p.step(x=df['update_time'], y=df['sum_profit'])

p = chart2.get_figure('main')
buy_df = df[df['order_side'] == "Buy"]
p.scatter(x=buy_df['update_time'], y=buy_df['order_price'], marker='triangle', size=10, color="#6699ff")

sell_df = df[df['order_side'] == "Sell"]
p.scatter(x=sell_df.index, y=sell_df['order_price'], marker='inverted_triangle', size=10, color="#ff8080")


p = chart2.new_figure('indicator', 150, 'indicator')

indicator = agent.get_indicator('diff_low')
p.line(x=indicator.index, y=indicator['value'], color='#ff8080')

indicator = agent.get_indicator('diff_high')
p.line(x=indicator.index, y=indicator['value'], color='#80ff80')

indicator = agent.get_indicator('range')
p.line(x=indicator.index, y=indicator['value'], color='#8080ff')


chart2.show()

Loading BokehJS ...

NameError: name 'df' is not defined

In [ ]:
agent.get_indicator_names()

['diff_low', 'diff_high', 'range']

In [ ]:
indicator

,diff_low
timestamp,
NaT,"[1669910400000000, 194.09000000000015]"
NaT,"[1669917600000000, 58.11999999999898]"
NaT,"[1669924800000000, 68.33000000000175]"
NaT,"[1669932000000000, 106.88000000000102]"
NaT,"[1669939200000000, 3.2400000000016007]"
...,...
NaT,"[1670601600000000, 70.20000000000073]"
NaT,"[1670608800000000, 30.340000000000146]"
NaT,"[1670616000000000, 44.919999999998254]"


In [11]:
agent.get_indicator('diff_low')

ValueError: Must pass 2-d input. shape=()

In [19]:
buy_df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-04 02:00:00+00:00,2022-12-04 02:10:00.004000+00:00,0000-0003,0,Buy,True,Expire,0.0,0.0,0.0,0.0,...,16888.07,0.01,168.8807,0.0,0.0,0.0,0.0,Open Long,0.0,0.0
2022-12-05 02:00:00+00:00,2022-12-05 02:10:00.152000+00:00,0000-0004,0,Buy,True,Expire,0.0,0.0,0.0,0.0,...,17109.65,0.01,171.0965,0.0,0.0,0.0,0.0,Open Long,0.0,0.0
2022-12-05 06:00:00+00:00,2022-12-05 06:10:00.193000+00:00,0000-0005,0,Buy,True,Expire,0.0,0.0,0.0,0.0,...,17200.68,0.01,172.0068,0.0,0.0,0.0,0.0,Open Long,0.0,0.0
2022-12-08 20:00:00+00:00,2022-12-08 20:10:00.102000+00:00,0000-0008,0,Buy,True,Expire,0.0,0.0,0.0,0.0,...,16967.98,0.01,169.6798,0.0,0.0,0.0,0.0,Open Long,0.0,0.0


In [20]:
sell_df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-02 14:00:00+00:00,2022-12-02 14:10:00.053000+00:00,0000-0001,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,16997.89,0.01,169.9789,0.0,0.0,0.0,0.0,Open Short,0.0,0.0
2022-12-04 00:00:00+00:00,2022-12-04 00:10:00.020000+00:00,0000-0002,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,16949.21,0.01,169.4921,0.0,0.0,0.0,0.0,Open Short,0.0,0.0
2022-12-05 16:00:00+00:00,2022-12-05 16:10:00.066000+00:00,0000-0006,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,17234.33,0.01,172.3433,0.0,0.0,0.0,0.0,Open Short,0.0,0.0
2022-12-07 08:00:00+00:00,2022-12-07 08:10:00.030000+00:00,0000-0007,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,17005.57,0.01,170.0557,0.0,0.0,0.0,0.0,Open Short,0.0,0.0
2022-12-09 14:00:00+00:00,2022-12-09 14:10:00.314000+00:00,0000-0009,0,Sell,True,Expire,0.0,0.0,0.0,0.0,...,17242.62,0.01,172.4262,0.0,0.0,0.0,0.0,Open Short,0.0,0.0


In [21]:
rbot.init_debug_log()
bn.download(10)

NameError: name 'bn' is not defined

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time

In [ ]:
df['order_side'] == rbot.OrderSide.Buy

create_time
2022-12-01 14:00:00+00:00    False
2022-12-02 14:00:00+00:00     True
2022-12-04 02:00:00+00:00    False
2022-12-04 02:00:00+00:00    False
2022-12-05 16:00:00+00:00     True
2022-12-05 16:00:00+00:00     True
Name: order_side, dtype: bool

In [68]:
indicators = {}

indicators['a'] = []

indicators['a'].append((1, 1.0))
indicators['a'].append((2, 2.0))
    

In [69]:
indicators['a']

[(1, 1.0), (2, 2.0)]

In [22]:
class A:
    def __init__(self):
        self._indicators = {}

    def key_in_indicators(self, key):
        if key in self._indicators:
            return True
        else:
            return False

    def get_indicator(self, key):
        if not self.key_in_indicators(key):
            return None

        return self._indicators[key]
        

    def set_indicator(self, key, time, val):
        if not self.key_in_indicators(key):
            self._indicators[key] = []
        
        self._indicators[key].append([time, val])
        
    def get_indicator_names(self):
        names = []
        for k in self._indicators:
            names.append(k)
        
        return names
       

In [23]:
a = A()

In [24]:
a.get_indicator('a')

In [26]:
a.set_indicator("b", 5, 4)

In [27]:
n = a.get_indicator_names()

In [29]:
ind = a.get_indicator('b')


In [32]:
import numpy as np
ind_n = np.array(ind)

In [33]:
ind_n

array([[4, 3],
       [5, 4]])

In [34]:
import pandas as pd

In [37]:
aa = pd.DataFrame(ind_n, columns=['timestamp', 'v'])

In [38]:
        aa['timestamp'] = pd.to_datetime((aa["timestamp"]), utc=True, unit='us')
        aa = aa.set_index('timestamp')

In [11]:
fig = OrderedDict()

In [14]:
'a' in fig.keys()

False

In [1]:
for i in range(0, 67172864):
